In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

In [81]:
# HREF SCRAPER
def get_urls(n_pages, section):
    print('getting urls')
    urls = []
    c = 0
    for page in range(n_pages+1):
        try:
            soup = BeautifulSoup(get('https://www.dailytexanonline.com/section/{0}?page={1}'.format(section, page)).text,'lxml')
            for i in soup.findAll('h2', {'class':'story-title'}):
                urls.append(i.find('a')['href'])
            if c % 20 == 0:
                print('{:.2f}'.format(c/n_pages), end=' ')
            c += 1
        except AttributeError:
            print(page)
    return urls

In [91]:
# ARTICLE SCRAPER
def get_articles(urls):
    print('getting articles')
    articles = {}
    c = 0
    for h in urls:
        try:
            soup = BeautifulSoup(get('https://www.dailytexanonline.com'+h).text,'lxml')
            auth = soup.find('div',{'id':'author-name'})
            articles[h] = dict(
                text = soup.find('div',{'id':'article-body'}).text,
                author = auth.text,
                author_link = auth.find('a')['href'],
                date = soup.find('time',{'id':'article-published'}).text,
                title = soup.find('h1',{'id':'page-title'}).text,
    #             these two don't load in time
        #         likes = soup.find('span',{'id':'u_0_1'}).text,
        #         comments = soup.find('span',{'id':'comment-count'}).text,
            )
        except:
            print(h)
        if c % 20 == 0:
            print('{:.2f}'.format(c/len(urls)), end=' ')
        c += 1
    articles = pd.DataFrame.from_dict(articles, orient='index')
    articles.date = pd.to_datetime(articles.date.str.lstrip('Published on '), format='%B %d, %Y at %I:%M %p')
    articles.text = articles.text.str.replace('\\xa0',' ').str.replace('\\n',' ')
    articles.author = articles.author.str.lstrip('By ')
    articles.title = articles.title.str.strip()
    return articles

In [92]:
get_articles(get_urls(178,'campus')).to_pickle('data/campus.p')

getting urls
0.00 0.11 0.22 0.34 0.45 0.56 0.67 0.79 0.90 getting articles
0.00 0.01 0.02 0.03 0.04 0.06 /2017/05/07/former-university-president-peter-flawn-passes-away
0.07 0.08 0.09 0.10 0.11 0.12 0.13 0.15 0.16 0.17 0.18 0.19 0.20 0.21 0.22 0.24 0.25 0.26 0.27 0.28 0.29 0.30 0.31 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.40 0.41 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.50 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.70 0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.80 /2015/09/24/ut-system-to-launch-sexual-assualt-survey-in-october
0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.89 0.90 0.91 0.92 0.93 0.94 0.95 0.96 0.98 0.99 1.00 

In [52]:
data = pd.read_pickle('data/opinion.p')
data.text = data.text.str.strip()
data.author = data.author.str.lower()
# data = data[data.author.isin(data.author.value_counts().head(20).index)]

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(stop_words='english', strip_accents='unicode').fit(data.text)
w = v.transform(data.text)

In [50]:
from sklearn.naive_bayes import MultinomialNB
y = LabelEncoder().fit_transform(data.author.str.lower())
m = MultinomialNB(alpha=.1).fit(w, y)

In [51]:
m.score(w, y)

0.97190293742017875

In [54]:
from sklearn.cluster import KMeans
k = KMeans(n_jobs=-1, verbose=1, random_state=42).fit(w)

Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
Iteration  0, inertia 4730.641
Iteration  0, inertia 4743.456
Iteration  0, inertia 4683.911
Iteration  0, inertia 4730.312
Iteration  0, inertia 4740.505
Iteration  0, inertia 4724.488
Iteration  0, inertia 4734.731
Iteration  0, inertia 4729.638
Iteration  1, inertia 2449.850
Iteration  1, inertia 2447.293
Iteration  1, inertia 2456.910
Iteration  1, inertia 2450.733
Iteration  1, inertia 2450.318
Iteration  1, inertia 2452.134
Iteration  1, inertia 2457.944
Iteration  1, inertia 2456.176
Iteration  2, inertia 2439.075
Iteration  2, inertia 2438.401
Iteration  2, inertia 2438.472
Iteration  2, inertia 2438.705
Iteration  2, inertia 2445.336
Iteration  2, inertia 2440.616
Iteration  2, inertia 2445.380
Iteration  2, inertia 2444.665
Iteration  3, inertia 2435.751
Iteration  3, inertia 2434.428
It

In [58]:
cc = pd.DataFrame(data=k.cluster_centers_, columns=v.vocabulary_).T
cc[0].sort_values(ascending=False)

spurning          0.109158
harmfully         0.077428
nepotistic        0.076215
ingloriously      0.066014
inefficiency      0.061617
constituency      0.052348
brashly           0.048730
1462              0.046882
barrel            0.045194
demonstrates      0.041288
devoting          0.040403
surcharge         0.039286
gnarly            0.036088
wooing            0.032325
erg               0.031470
sweetheart        0.030562
356               0.030233
foodstuffs        0.030133
surrendering      0.029976
cassell           0.028429
modulators        0.027308
ny                0.026300
crudely           0.024046
boss              0.023613
120               0.021996
discomfort        0.021676
courtrooms        0.021377
template          0.021295
campuswide        0.021219
eliciting         0.020514
                    ...   
devon             0.000000
dysfunctional     0.000000
sanger            0.000000
contextualizes    0.000000
circumvented      0.000000
borrows           0.000000
e

In [60]:
data['cluster'] = k.predict(w)

In [70]:
(data[data.cluster==0].author.value_counts()/data.author.value_counts()).dropna().sort_values(ascending=False)

matt offill                           1.000000
mitchell hughes ralph                 1.000000
ezra siegel                           1.000000
evan berkowitz                        1.000000
juan carlos delafuente                1.000000
ou daily                              1.000000
marc nestenius                        1.000000
david leffler                         1.000000
garrett callahan                      1.000000
curry shoff                           1.000000
christian corona                      1.000000
the oklahoma daily editorial board    1.000000
thomas hunt, anne mueller             1.000000
chad markey                           1.000000
nick castillo                         1.000000
travis adams                          0.666667
jasmine c. johnson                    0.666667
jori epstein                          0.615385
daley epstein                         0.500000
matthew gil                           0.500000
patrick st. pierre                    0.500000
zachary adams